# graph

> TODO fill in description

In [1]:
#| default_exp graph

In [2]:
#| hide
from nbdev.showdoc import *; import nbdev; nbdev.nbdev_export()

In [3]:
#|export
import asyncio
from typing import Type, Optional, Union, Any
from types import MappingProxyType
from enum import Enum
import copy

import fbdev
from fbdev.utils import AttrContainer
from fbdev.component import PortType, PortSpec, PortTypeSpec, PortSpecCollection

In [4]:
#|hide
from fbdev.component import BaseComponent

In [5]:
#|hide
show_doc(fbdev.graph.EdgeSpec)

---

### EdgeSpec

>      EdgeSpec (maxsize:int=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [6]:
#|export
class EdgeSpec:
    def __init__(self,
                 maxsize:int=None):
        self._id = None
        self._parent_graph = None
        
        self._tail_target = None
        self._tail_port_type = None
        self._tail_port_name = None
        
        self._head_target = None
        self._head_port_type = None
        self._head_port_name = None
        
        self.maxsize = maxsize
        
    @property
    def id(self): return self._id
    @property
    def attached(self): return self.tail_target is not None or self.head_target is not None
    
    @property
    def tail_is_connected_to_graph(self): return self.tail_target is None and self.tail_port_type is not None
    @property
    def head_is_connected_to_graph(self): return self.head_target is None and self.head_port_type is not None
    @property
    def is_connected_to_graph(self): return self.tail_is_connected_to_graph or self.head_is_connected_to_graph
    
    @property
    def tail_target(self): return self._tail_target
    @property
    def tail_port_type(self): return self._tail_port_type
    @property
    def tail_port_name(self): return self._tail_port_name
    
    @property
    def head_target(self): return self._head_target
    @property
    def head_port_type(self): return self._head_port_type
    @property
    def head_port_name(self): return self._head_port_name
    
    @property
    def tail_is_connected(self): return self.tail_port_type is not None
    @property
    def head_is_connected(self): return self.head_port_type is not None

    
    def _set_head(self, target: Optional['NodeSpec'], port_type: PortType, port_name: str):
        self._head_target = target
        self._head_port_type = port_type
        self._head_port_name = port_name
    
    def _clear_head(self):
        self._head_target = None
        self._head_port_type = None
        self._head_port_name = None
    
    def _set_tail(self, target: Optional['NodeSpec'], port_type: PortType, port_name: str):
        self._tail_target = target
        self._tail_port_type = port_type
        self._tail_port_name = port_name
    
    def _clear_tail(self):
        self._tail_target = None
        self._tail_port_type = None
        self._tail_port_name = None

In [7]:
#|hide
show_doc(fbdev.graph.NodeSpec)

---

### NodeSpec

>      NodeSpec (component_type)

*Initialize self.  See help(type(self)) for accurate signature.*

In [8]:
#|export
class NodeSpec:
    def __init__(self, component_type):
        self._id = None
        self._parent_graph = None
        self._component_type = component_type
        self._port_connections = AttrContainer({}, obj_name="Node.port_connections")
        for port_type_spec in self._component_type.port_specs:
            self.port_connections._set(port_type_spec.port_type, AttrContainer(obj_name=f"NodeSpec.port_connections.{port_type_spec.port_type}", dtype=EdgeSpec))
            for port_name in port_type_spec:
                self.port_connections[port_type_spec.port_type]._set(port_name, None)
        
    @property
    def attached_to_graph(self): return self._parent_graph is not None
    @property
    def id(self): return self._id
    @property
    def component_type(self): return self._component_type
    @property
    def port_connections(self): return self._port_connections

In [9]:
#|hide
show_doc(fbdev.graph.Graph)

---

### Graph

>      Graph ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [10]:
#|export
class Graph(NodeSpec):
    def __init__(self):
        self._nodes = {}
        self._edges = {}
        
        self._graph_port_specs = PortSpecCollection()
        # port attributes: port_name, dtype, data_validator
        #  if config: default
        self._graph_port_connections = AttrContainer({ port_type.label : {} for port_type in PortType }, obj_name="Graph.graph_port_connections")
        
    @property
    def nodes(self): return MappingProxyType(self._nodes)
    @property
    def edges(self): return MappingProxyType(self._edges)
    @property
    def graph_port_specs(self):
        read_only_attrs = {k:MappingProxyType(v) for k,v in self._graph_port_specs._attrs.items()}
        return AttrContainer(read_only_attrs, obj_name=self._graph_port_specs._obj_name)
    @property
    def graph_port_connections(self):
        read_only_attrs = {k:MappingProxyType(v) for k,v in self._graph_port_connections._attrs.items()}
        return AttrContainer(read_only_attrs, obj_name=self._graph_port_connections._obj_name)
        
    def add_graph_port(self, port_type: PortType, port_name: str, dtype=Optional[Type], data_validator=None, default=None, default_is_None=False):
        if port_name in self._graph_port_specs[port_type.label]:
            raise RuntimeError(f"Port {port_type.label}.{port_name} already exists.")
        self._graph_port_specs[port_type.label][port_name] = PortSpec(port_name, dtype, data_validator, default, default_is_None)
        self._graph_port_connections[port_type.label][port_name] = None
        
    def remove_graph_port(self, port_type: PortType, port_name: str):
        if port_name not in self._graph_port_specs[port_type.label]:
            raise RuntimeError(f"Port {port_type.label}.{port_name} does not exist.")
        edge = self._graph_port_connections[port_type.label][port_name]
        if edge is not None:
            self.disconnect_edge_from_graph_port(port_type, port_name)
        del self._graph_port_specs[port_type.label][port_name]
        del self._graph_port_connections[port_type.label][port_name]
        
    def _check_node(self, node: NodeSpec):
        if node._parent_graph is None: self.add_node(node)
        if node._parent_graph != self: raise RuntimeError("Node is already in a graph.")
    def _check_edge(self, edge: EdgeSpec):
        if edge._parent_graph is None: self.add_edge(edge)
        if edge._parent_graph != self: raise RuntimeError("Edge is already in a graph.")
        
    def connect_edge_to_node(self, node, port_type: PortType, port_name: str, edge):
        if type(node) != NodeSpec: node = self.nodes[node]
        if type(edge) != EdgeSpec: edge = self.edges[edge]
        self._check_node(node)
        self._check_edge(edge)
        if node.port_connections[port_type.label][port_name] is not None: raise RuntimeError(f"Edge already connected to port {port_type.label}.{port_name}.")
        node.port_connections[port_type.label][port_name] = edge
        if port_type.is_input: edge._set_head(node, port_type, port_name)
        else: edge._set_tail(node, port_type, port_name)
        
    def disconnect_edge_from_node(self, node: Union[str, NodeSpec], port_type: PortType, port_name: str):
        if type(node) != NodeSpec: node = self.nodes[node]
        self._check_node(node)
        edge = node.port_connections[port_type.label][port_name]
        node.port_connections[port_type.label][port_name] = None
        if port_type.is_input: edge._clear_head()
        else: edge._clear_tail()
        
    def connect_edge_to_graph_port(self, port_type: PortType, port_name: str, edge:EdgeSpec):
        if type(edge) != EdgeSpec: edge = self.edges[edge]
        self._check_edge(edge)
        if self._graph_port_connections[port_type.label][port_name] is not None: raise RuntimeError(f"Edge already connected to port {port_type.label}.{port_name}.")
        self._graph_port_connections[port_type.label][port_name] = edge
        if port_type.is_input: edge._set_head(None, port_type, port_name)
        else: edge._set_tail(None, port_type, port_name)
        
    def disconnect_edge_from_graph_port(self, port_type: PortType, port_name: str):
        edge = self._graph_port_connections[port_type.label][port_name]
        self._graph_port_connections[port_type.label][port_name] = None
        if port_type.is_input: edge._clear_head()
        else: edge._clear_tail()
        
    def add_node(self, node, id=None):
        if node._parent_graph is not None: raise RuntimeError("Node is already in a graph.")
        node._id = id or len(self.nodes)
        if node._id in self.nodes: raise RuntimeError(f"Node id {node._id} already exists.")
        node._parent_graph = self
        self._nodes[node.id] = node
        
    def add_edge(self, edge: EdgeSpec, id=None):
        if edge._parent_graph is not None: raise RuntimeError("Edge is already in a graph.")
        edge._id = id or len(self.edges)
        edge._parent_graph = self
        self._edges[edge.id] = edge
        
    def remove_node(self, node: Union[str, NodeSpec]):
        if type(node) != NodeSpec: node = self.nodes[node]
        if node.graph != self: raise RuntimeError("Node is not in this graph.")
        node._graph = None
        for port_type in node.port_connections:
            for port_name in node.port_connections[port_type]:
                if node.port_connections[port_type][port_name] is not None:
                    self.disconnect_edge(node, PortType(port_type.upper()), port_name)
        node.port_connections._attrs.clear()
        
    def remove_edge(self, edge: EdgeSpec):
        if type(edge) != EdgeSpec: edge = self.edges[edge]
        if edge._parent_graph != self or edge.id not in self.edges: raise RuntimeError("Edge is not in this graph.")
        edge._id = None
        edge._parent_graph = None
        del self._edges[edge.id]
        
        if edge.tail_is_connected:
            if edge.tail_is_connected_to_graph:
                self.disconnect_edge_from_graph_port(edge.tail_port_type, edge.tail_port_name)
            else:
                self.disconnect_edge_from_node(edge.tail_target, edge.tail_port_type, edge.tail_port_name)
        if edge.head_is_connected:
            if edge.head_is_connected_to_graph:
                self.disconnect_edge_from_graph_port(edge.head_port_type, edge.head_port_name)
            else:
                self.disconnect_edge_from_node(edge.head_target, edge.head_port_type, edge.head_port_name)
            
    def copy(self):
        g = Graph()
        
        for node in self.nodes.values():
            g.add_node(NodeSpec(node.component_type), id=node.id)
            
        for edge in self.edges.values():
            g.add_edge(EdgeSpec(maxsize=edge.maxsize), id=edge.id)
            
            if edge.tail_is_connected:
                if edge.tail_is_connected_to_graph:
                    g.connect_edge_to_graph_port(edge.tail_port_type, edge.tail_port_name, edge.id)
                else:
                    g.connect_edge_to_node(edge.tail_target.id, edge.tail_port_type, edge.tail_port_name, edge.id)
            if edge.head_is_connected:
                if edge.head_is_connected_to_graph:
                    g.connect_edge_to_graph_port(edge.head_port_type, edge.head_port_name, edge.id)
                else:
                    g.connect_edge_to_node(edge.head_target.id, edge.head_port_type, edge.head_port_name, edge.id)
                    
        return g
    

In [11]:
g = Graph()
node = NodeSpec(BaseComponent)
g.add_node(node)
assert node in g.nodes.values()
assert node.id == 0
g.add_node(NodeSpec(BaseComponent))
g.add_node(NodeSpec(BaseComponent))
assert len(g.nodes) == 3
edge = EdgeSpec()
g.add_edge(edge)
assert edge in g.port_connections.values()

In [12]:
#|hide
show_doc(fbdev.graph.ReadonlyGraph)

---

### ReadonlyGraph

>      ReadonlyGraph (graph:fbdev.graph.Graph)

*Initialize self.  See help(type(self)) for accurate signature.*

In [13]:
#|export
class ReadonlyGraph:
    def __init__(self, graph: Graph):
        self._graph = graph
        
    @property
    def nodes(self): return self._graph.nodes
    @property
    def edges(self): return self._graph.port_connections
    @property
    def graph_port_specs(self): return self._graph.graph_port_specs
    @property
    def graph_port_connections(self): return self._graph.graph_port_connections
    
    def copy(self) -> Graph: return self._graph.copy()